### Test of codes using the Franke function

First import necessary modules:

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import os, sys
#make sure we can import modules from the same folder where the file is
import os, sys
sys.path.append(os.path.join(os.path.dirname(__file__), "../"))
sys.path.append(os.path.dirname(__file__))
import plotting_functions as plotfnc
import franke_fit as ff
import utils

ModuleNotFoundError: No module named 'plotting_functions'

Generation of the dataset:

In [ ]:
#this list will be used to loop through different optimization methods
methods = [OLS, Ridge, Lasso]

#seed RNG to reproduce results
np.random.seed(133)

# Make dataset.
Nx_ = 16
Ny_ = 16

#maximum polynomial degree
maxdeg = 12

#generate x,y data from uniform distribution in [0,1]
x__ = np.random.rand(Nx_, 1)
y__ = np.random.rand(Ny_, 1)
x_, y_ = np.meshgrid(x__,y__)
#generate target data adding gaussian noise
z_ = (utils.FrankeFunction(x_, y_) + 0.1*np.random.randn(Nx_,Ny_)).reshape(-1,1)

Study of MSE and $R^2$ score as function of the polynomial degree, using OLS

In [ ]:
degrees_list, MSE_train_list, MSE_test_list, _, _, _, R2_train_list, R2_test_list, _ \
= Solver(x_, y_, z_, Nx_, Ny_, OLS, lamb = 0, useBootstrap=False, useCrossval=False, maxdegree = maxdeg)

plotfnc.MSE_R2_plot(degrees_list, MSE_train_list, MSE_test_list, R2_train_list, R2_test_list, savefig = True)